# word2vec
word2vecにはCBOWとskip-gramが存在。  
CBOWはコンテキストから単語を推測、skip-gramは単語から周囲のコンテキストを推測。


### CBOW
CBOWモデルはコンテキストからターゲットを推測するモデル。  
（前後1単語から推測する場合は）入力層が二つ、中間層が一つ、出力層で出来ている。  
出力層は各単語のスコアを計算し、最終的にsoftmax関数を用いてターゲットとなる単語を選ぶ。  
損失関数は
$$ L = -\frac{1}{T}\sum^T_{t=1}\log{P(w_t | w_{t-1}, w_{t+1})}$$


### skip-gram
CBOWモデルを逆転させたような構造。  
前後1単語を予測する場合は次のような損失関数
$$ L= -\frac{1}{T}\sum^T_{t=1}(\log{P(w_{t-1}|w_t)+log{P(w_{t+1}|w_t)}}) $$


参考url
- https://qiita.com/g-k/items/69afa87c73654af49d36


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import random
import pandas as pd

df = pd.read_csv('dataset/token.csv')
df = df[1:40]
sentences = []
for text in df['article']:
    text_list = text.split(' ')
    sentences.append(text_list)

from gensim.models import Word2Vec
model = Word2Vec(sentences,  sg=1, vector_size=100, window=5, min_count=1)

In [12]:
for word, similarity in model.wv.most_similar('家族'):
    print(f"{word}: {similarity}")


行く: 0.9988073706626892
通勤: 0.9986675381660461
分: 0.9986549019813538
返信: 0.9986339807510376
なかなか: 0.998598039150238
支給: 0.9985608458518982
様子: 0.9985296130180359
位: 0.9985253214836121
初回: 0.9985184669494629
理解: 0.9985105991363525


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"現在使用中のデバイス: {device}")


現在使用中のデバイス: cpu


/home/st6322041/anaconda3/envs/ML/lib/python3.9/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [14]:
results = model.wv.most_similar(positive=['人生'], negative=['幸福'])
for word, similarity in results:
    print(f"{word}: {similarity:.4f}")


KeyError: "Key '幸福' not present in vocabulary"

In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import random
import pandas as pd

df = pd.read_csv('dataset/token.csv')
df.head
# sentences = []
# for text in df:
#     text_list = text.split(' ')
#     sentences.append(text_list)

# from gensim.models import Word2Vec
# model = Word2Vec(sentences,  sg=1, size=100, window=5, min_count=1)

<bound method NDFrame.head of          0  \
0        1   
1        2   
2        3   
3        4   
4        5   
...    ...   
7361  7362   
7362  7363   
7363  7364   
7364  7365   
7365  7366   

     もうすぐ ジューン ・ ブライド と 呼ば れる 6 月 。 独 女 の 中 に は 自分 の 式 は まだ な のに 呼ば れ て ばかり ...... という 「 お祝い 貧乏 」 状態 の 人 も 多い の で は ない だろ う か ? さらに 出席 回数 を 重ね て いく と 、 こんな お願い ごと を さ れる こと も 少なく ない 。 「 お願い が ある ん だ けど ...... 友人 代表 の スピーチ 、 やっ て くれ ない か な ?」 さて そんな とき 、 独 女 は どう 対応 し たら いい か ? 最近 だ と インターネット 等 で 検索 すれ ば 友人 代表 スピーチ 用 の 例文 サイト が たくさん 出 て くる ので 、 それら を 参考 に すれ ば 、 無難 な もの は 誰 でも 作成 できる 。 しかし 由利 さん ( 33 歳 ) は ネット を 参考 に し て 作成 し た ものの 「 これ で 本当に いい の か 不安 でし た 。 一人暮らし な ので 聞か せ て 感想 を いっ て くれる 人 も い ない し 、 か と いっ て 他 の 友人 に わざわざ 聞か せる の も どう か と 思う し ......」 という こと で 活用 し た の が 、 なんと インターネット の 悩み 相談 サイト に 。 そこ に 作成 し た スピーチ 文 を 掲載 し 「 これ で 大丈夫 か 添削 し て ください 」 と メッセージ を 送っ た という の で ある 。 「 一 晩 で 3 人 位 の 人 が 添削 し て くれ まし た よ 。 ちなみに 自分 以外 に も そういう 人 は たくさん い て 、 その 相談 サイト に は 同じ よう に 添削 を お願い する 投

In [ ]:
corpus = "You say goodbye and I say hello"
# print(corpus.lower())
corpus = corpus.lower().split()
# print(corpus)
# print(set(corpus))
# print(enumerate(set(corpus)))
# for i, item in enumerate(set(corpus)):
#     print(i, item)
# 単語をIDに変換
word2idx = {word: idx for idx, word in enumerate(set(corpus))} # wordからindexを取得するリスト
print(word2idx)
idx2word = {idx: word for word, idx in word2idx.items()} # indexからwordを取得するリスト
print(word2idx.items())
print(idx2word)
vocab_size = len(word2idx)

# 単語ID列
corpus_ids = [word2idx[w] for w in corpus]
print("\"You say goodbye and I say hello\" is transformed as", corpus_ids)


0 say
1 i
2 hello
3 you
4 goodbye
5 and
{'say': 0, 'i': 1, 'hello': 2, 'you': 3, 'goodbye': 4, 'and': 5}
dict_items([('say', 0), ('i', 1), ('hello', 2), ('you', 3), ('goodbye', 4), ('and', 5)])
{0: 'say', 1: 'i', 2: 'hello', 3: 'you', 4: 'goodbye', 5: 'and'}
"You say goodbye and I say hello" is transformed as [3, 0, 4, 5, 1, 0, 2]


In [4]:
def generate_training_data(corpus_ids, window_size=2):
    pairs = []
    for i, target in enumerate(corpus_ids):
        for j in range(-window_size, window_size + 1):
            if j == 0 or i + j < 0 or i + j >= len(corpus_ids):
                continue
            context = corpus_ids[i + j]
            pairs.append((target, context))
    return pairs

pairs = generate_training_data(corpus_ids)


In [30]:
class Word2Vec(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.input_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.output_embeddings = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, center_words, context_words, negative_words):
        center_embeds = self.input_embeddings(center_words)
        pos_embeds = self.output_embeddings(context_words)
        neg_embeds = self.output_embeddings(negative_words)

        pos_score = torch.sum(center_embeds * pos_embeds, dim=1)
        pos_loss = torch.log(torch.sigmoid(pos_score))

        neg_score = torch.bmm(neg_embeds, center_embeds.unsqueeze(2)).squeeze(2)
        neg_loss = torch.log(torch.sigmoid(-neg_score), dim=1)

        return -(pos_loss + neg_loss).mean()


In [31]:
embedding_dim = 10
model = Word2Vec(vocab_size, embedding_dim)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# ネガティブサンプリング関数
def get_negative_samples(pos_word, vocab_size, num_neg=5):
    neg_samples = []
    while len(neg_samples) < num_neg:
        w = random.randint(0, vocab_size - 1)
        if w != pos_word:
            neg_samples.append(w)
    return neg_samples

# 学習
for epoch in range(100):
    total_loss = 0
    for center, context in pairs:
        center_tensor = torch.tensor([center])
        context_tensor = torch.tensor([context])
        negative_tensor = torch.tensor([get_negative_samples(context, vocab_size)]).unsqueeze(0)

        loss = model(center_tensor, context_tensor, negative_tensor)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss:.4f}")


RuntimeError: batch1 must be a 3D tensor

In [8]:
embeddings = model.input_embeddings.weight.data
for word, idx in word2idx.items():
    print(f"{word}: {embeddings[idx]}")


say: tensor([-0.2802,  0.8585,  0.8102,  0.5385, -0.0954,  1.1551,  0.2458, -1.8385,
        -0.3713, -1.2752])
i: tensor([ 0.2179,  0.2700,  0.7094,  1.1787, -0.3879,  0.6546, -0.9337,  0.7479,
        -0.1273, -0.7252])
hello: tensor([-1.4959, -0.6925, -0.4337, -1.3229, -0.2961, -0.3127,  1.0085,  0.4916,
         0.7628, -1.1374])
you: tensor([-0.1185, -0.3650, -0.9618, -1.0986,  0.5777,  0.4606,  0.1865,  0.1715,
        -0.1041,  1.1317])
goodbye: tensor([ 0.0953, -1.0774, -0.5591,  0.3436, -0.1011,  0.8020, -0.0540, -0.0117,
         0.1995,  0.7604])
and: tensor([ 0.2971, -0.3863, -0.0771, -1.1469,  0.8770, -1.3631, -2.7957,  0.3792,
        -0.2224,  0.0461])
